# CX Circuits

Changing working directory from `./notebooks/` to `./`, in order to import the Python packages defined in the repository.

In [ ]:
import os
DEBUG = True
try:
    print("Original working directory: %s"%str(original_wd)) # type: ignore
    """
        You only get here if you---by mistake or otherwise---are re-running this cell, 
        in which case the working should not be changed again.
    """
except NameError:
    original_wd = os.getcwd()
    os.chdir('../')
print("Current working directory: %s"%str(os.getcwd()))

General purpose imports:

In [ ]:
import matplotlib.pyplot as plt
# import qiskit

Some utility code to display HTML elements and images/figures side-by-side:

In [ ]:
from io import BytesIO
from typing import Union
from IPython.display import Image, HTML # type: ignore
def figure_to_image(fig: plt.Figure, fmt: str = "png") -> Image:
    """ Converts a Matplotlib figure to a PNG IPython Image. """
    buffer = BytesIO()
    fig.savefig(buffer, format=fmt)
    buffer.seek(0)
    image_png = buffer.getvalue()
    buffer.close()
    return Image(image_png, format=fmt, embed=True)
def side_by_side(*elements: Union[str, Image]) -> HTML:
    """
        Returns an HTML Div element with the given elements
        displayed side by side. Accepts raw HTML code or
        IPython Image objects.
    """
    html = f"<div style='display:flex; align-items: center;'>"
    for el in elements:
        if isinstance(el, str):
            html += el
        elif isinstance(el, Image):
            html += f"<img src='data:image/png;base64,{el._repr_png_()}'/>"
    html += f"</div>"
    return HTML(html)

## Conjugation by CX Circuits

One of the optimisation techniques for a circuit $\mathcal{C}$ of mixed ZX phase gadgets proceeds through conjugation of the circuit by a suitably chosen block $U$ of CX gates, obtaining another circuit $U \circ \mathcal{C}' \circ U^\dagger$ (hopefully simpler overall).

These CX circuits are constructed and manipulated using the classes `CXCircuit` and `CXCircuitLayer`. Read-only views of such circuits are made available by the respective `as_readonly` properties, through the `CXCircuitView` and `CXCircuitLayerView` classes.

In [ ]:
from pauliopt.phase import CXCircuit, CXCircuitLayer

## CX Circuit Layers

A single layer of a CX circuit is constructed with `CXCircuitLayer` from a topology and a collection of CX gates, specified as pairs `(ctrl, trgt)` of control and target qubits (in `CXCircuitLayer.topology.qubits`).
It uses a `Matching` instance internally to keep track of which qubit pairs are covered by some gate.

In [ ]:
from pauliopt.topologies import Topology
topology = Topology.grid(3,3)
layer = CXCircuitLayer(topology, [[0, 1], [2, 5], [6, 7]])

CX circuit layers exposes the underlying topology, number of gates and gates through suitably named read-only properties. Note that the `CXCircuitLayer.gates` property involves the creation of a fresh frozen-set at every call, since gates are stored internally in a different data structure.   

In [ ]:
print(f"{layer.topology = }")
print(f"{layer.num_gates = }")
print(f"{layer.gates = }")

CX circuit layers expose information about the gate incident to any given qubit (if any), through the `CXCircuitLayer.incident(qubit)` method.

In [ ]:
for q in layer.topology.qubits:
    print(f"Gate on qubit {q}: {layer.incident(q)}")

CX circuit layers can be drawn using the `CXCircuitLayer.draw(**kwargs)` method. The allowed keyword arguments are those of `Topology.draw(**kwargs)`, plus `zcolor` and `xcolor` to control the colours for the Z and X dots in CX gates (analogous to `PhaseCircuit.to_svg(**kwargs)`).  

In [ ]:
layer.draw(figsize=(4,4))

## CX Gate Flipping

CX circuit layers are modified by "flipping" CX gates, which can mean either:

- removing a CX gate currently in the layer;
- adding a CX gate to the layer on a pair of qubits, as long as neither qubit has an incident CX gate already.

This guarantees that all CX gates in a layer can be performed simultaneously on the given qubit topology.

The list of CX gates that can be flipped in a layer is exposed by the `CXCircuitLayer.flippable_cxs` property.

In [ ]:
print(f"Flippable CX gates: {layer.flippable_cxs}")
layer.draw(figsize=(3,3))

The `CXCircuitLayer.is_cx_flippable(ctrl, trgt)` boolean property can be used to check whether the CX gate with given control and target is flippable.

In [ ]:
print(f"{layer.is_cx_flippable(4, 3) = }") # no incident gates on qubits
print(f"{layer.is_cx_flippable(2, 5) = }") # gate in circuit
print(f"{layer.is_cx_flippable(1, 0) = }") # incident gate on both qubits
print(f"{layer.is_cx_flippable(7, 4) = }") # incident gate on one qubit
print(f"{layer.is_cx_flippable(1, 2) = }") # incident gates on both qubits
layer.draw(figsize=(3,3))

A CX gate with given control and target can be flipped using the `CXCircuitLayer.flip_cx(ctrl, trgt)` method (not available on the read-only view `CXCircuitLayerView`). The following example shows a CX gate being introduced on a pair of qubits `{3, 4}` neither of which already has a CX gate incident on it.

In [ ]:
print("Before flipping (4,3):")
layer.draw(figsize=(3,3))
layer.flip_cx(4,3)
print("After flipping (4,3):")
layer.draw(figsize=(3,3))

The following example shows a CX gate being removed from a circuit:

In [ ]:
print("Before flipping (6,7):")
layer.draw(figsize=(3,3))
layer.flip_cx(6,7)
print("After flipping (6,7):")
layer.draw(figsize=(3,3))

The in-place right-shift operator `>>=` can also be used to flip CX gates in a layer, resulting in a more compact syntax.

In [ ]:
layer = CXCircuitLayer(topology, [[0, 1], [2, 5], [6, 7]])
layer >>= [4, 3], [6,7]
layer.draw(figsize=(3,3))

In fact, CX circuit layers can be created entirely with the in-place right-shift operator.

In [ ]:
layer = CXCircuitLayer(topology)
layer >>= [0, 1], [2, 5], [6, 7], [4, 3], [6,7]
layer.draw(figsize=(3,3))

## CX Circuits

CX Circuits are constructed with `CXCircuit` from a topology and a sequence of CX circuit layers.

In [ ]:
circuit = CXCircuit(topology, [
    CXCircuitLayer(topology, [[0, 1], [2, 5], [6, 7]]),
    CXCircuitLayer(topology, [[1, 2], [5, 4], [8, 7], [3, 6]]),
    CXCircuitLayer(topology, [[0, 1], [4, 3], [7, 6]]),
])

CX circuits are a sequence of CX circuit layers: they can be iterated, individual layers can be accessed and the number of layers can be obtained with `len()`.

In [ ]:
print(f"Number of layers in circuit: {len(circuit)}")
for layer_idx, layer in enumerate(circuit):
    print(f"Gates in layer {layer_idx}: {layer.gates}")
print()
print("Layer 0:")
circuit[0].draw(figsize=(3, 3))

## Drawing CX Circuits



In [ ]:
circuit.draw(figsize=(3,3))

The in-place right-shift operator `>>=` can also be used to append layers to a CX circuit, resulting in a more compact syntax.

In [ ]:
circuit = CXCircuit(topology)
circuit >>= CXCircuitLayer(topology, [[0, 1], [2, 5], [6, 7]])
circuit >>= CXCircuitLayer(topology, [[1, 2], [5, 4], [8, 7], [3, 6]])
circuit.draw(figsize=(3,3))

In fact, the sequences of gates in each layer can be passed directly to the operator, further improving legibility.

In [ ]:
circuit = CXCircuit(topology)
circuit >>= [[0, 1], [2, 5], [6, 7]]
circuit >>= [[1, 2], [5, 4], [8, 7], [3, 6]]
circuit.draw(figsize=(3,3))

Passing multiple layers to the operator on a line is allowed, but discouraged. If you do, make sure you understand that each layer should be passed as a list of gates (otherwise the gates will all be merged into a single layer, usually raising an error because some flips are invalid).

In [ ]:
circuit = CXCircuit(topology)
circuit >>= [[0, 1], [2, 5], [6, 7]], [[1, 2], [5, 4], [8, 7], [3, 6]]
circuit.draw(figsize=(3,3))

Example of an error when passing multiple layers all in one go:

In [ ]:
circuit = CXCircuit(topology)
try:
    circuit >>= [0, 1], [2, 5], [6, 7], [1, 2], [5, 4], [8, 7], [3, 6]
except Exception as e:
    print("Error:", e)